In [1]:
import pandas as pd
import math

# -----------------------------
# BOX DIMENSIONS (meters)
# -----------------------------
# Outer box (insulated)
outer_length = 0.6
outer_width  = 0.4
outer_height = 0.21
A_outer = 2*(outer_length*outer_width + outer_length*outer_height + outer_width*outer_height)

# Inner box (soil)
inner_length = 0.4
inner_width  = 0.3
inner_height = 0.12
V_soil = inner_length * inner_width * inner_height
A_inner = 2*(inner_length*inner_width + inner_length*inner_height + inner_width*inner_height)

# -----------------------------
# MATERIAL PROPERTIES
# -----------------------------
# Sheep wool insulation
k_wool = 0.04  # W/m·K

# Soil properties
rho_soil = 1400    # kg/m³
c_soil   = 850     # J/kg·K

# Optional water in inner box (liters)
water_litres = [0, 2, 5, 10]

# External convection (W/m²·K)
h_out_values = [5]

# Insulation thicknesses (meters)
insulation_thicknesses = [0.02, 0.05, 0.10]  # 20mm, 50mm, 100mm

# Baseline thermal mass for very small contents (kg·J/kg·K)
mass_equiv = 0.5
c_equiv = 1200

# -----------------------------
# CALCULATIONS
# -----------------------------
rows = []

# Heat capacity of soil
C_soil = V_soil * rho_soil * c_soil  # J/K

for h_out in h_out_values:
    for t in insulation_thicknesses:
        # Thermal resistance of insulation + convection
        R_cond = t / (k_wool * A_outer)
        R_conv = 1.0 / (h_out * A_outer)
        R_total = R_cond + R_conv
        
        for L in water_litres:
            mass_water = L * 1000 / 1000  # kg (liters to kg)
            if mass_water == 0:
                C_total = C_soil
            else:
                C_water = mass_water * 4186  # J/K
                C_total = C_soil + C_water
            
            # Time constant
            tau_s = R_total * C_total
            tau_h = tau_s / 3600.0
            t90_h = -tau_h * math.log(0.1)
            
            rows.append({
                "h_out (W/m2K)": h_out,
                "insulation (mm)": int(t*1000),
                "water (L)": L,
                "C_total (J/K)": int(C_total),
                "R_total (K/W)": round(R_total,4),
                "tau (s)": int(tau_s),
                "tau (hours)": round(tau_h,2),
                "t90 (hours)": round(t90_h,2)
            })

# Create DataFrame
df = pd.DataFrame(rows)

# Export CSV
df.to_csv("box_insulated_thermal_time_constants.csv", index=False)
print("CSV file created: soil_box_thermal_time_constants.csv")
print(df)

CSV file created: soil_box_thermal_time_constants.csv
    h_out (W/m2K)  insulation (mm)  water (L)  C_total (J/K)  R_total (K/W)  \
0               5               20          0          17136         0.7778   
1               5               20          2          25508         0.7778   
2               5               20          5          38066         0.7778   
3               5               20         10          58996         0.7778   
4               5               50          0          17136         1.6111   
5               5               50          2          25508         1.6111   
6               5               50          5          38066         1.6111   
7               5               50         10          58996         1.6111   
8               5              100          0          17136         3.0000   
9               5              100          2          25508         3.0000   
10              5              100          5          38066         3.0000  

In [9]:
import pandas as pd
import math

# -----------------------------
# BOX DIMENSIONS (meters)
# -----------------------------
# Outside box (soil only)
length = 0.6
width  = 0.4
height = 0.21
V_soil = length * width * height
A_surface = 2*(length*width + length*height + width*height)

# -----------------------------
# SOIL PROPERTIES
# -----------------------------
rho_soil = 1400  # kg/m³
c_soil   = 850   # J/kg·K

# -----------------------------
# CONVECTION CONDITIONS (W/m²·K)
# -----------------------------
h_out_values = [5, 10]  # calm / moderate wind

# -----------------------------
# CALCULATIONS
# -----------------------------
rows = []

# Heat capacity of soil
C_soil = V_soil * rho_soil * c_soil  # J/K

for h_out in h_out_values:
    # Thermal resistance is only convective since no insulation
    R_total = 1.0 / (h_out * A_surface)
    
    # Time constant
    tau_s = R_total * C_soil
    tau_h = tau_s / 3600.0
    t90_h = -tau_h * math.log(0.1)
    
    rows.append({
        "h_out (W/m2K)": h_out,
        "C_soil (J/K)": int(C_soil),
        "R_total (K/W)": round(R_total,4),
        "tau (s)": int(tau_s),
        "tau (hours)": round(tau_h,2),
        "t90 (hours)": round(t90_h,2)
    })

# Create DataFrame
df = pd.DataFrame(rows)

# Export CSV
df.to_csv("box_soil_only_thermal_time_constants.csv", index=False)
print("CSV file created: soil_only_box_time_constants.csv")
print(df)


CSV file created: soil_only_box_time_constants.csv
   h_out (W/m2K)  C_soil (J/K)  R_total (K/W)  tau (s)  tau (hours)  \
0              5         59975         0.2222    13327         3.70   
1             10         59975         0.1111     6663         1.85   

   t90 (hours)  
0         8.52  
1         4.26  
